In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd


import pandas as pd
import math
import numpy as np


import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster
from folium import IFrame
import ipywidgets
from folium.features import CustomIcon

## API GEOCODE
### Añadiendo el key ID para poder utilizar la API

In [2]:
load_dotenv()

True

In [3]:
key = os.getenv("key")

### Función que va a convertir tu direccion en latitud y longitud

In [4]:
def geocode(calle):
    """
    Saca las coordenadas de una dirección que le des.
    """
    data = requests.get(f"https://geocode.xyz/{calle}?json=1").json()
    try:
        casa = [float(data["latt"]),float(data["longt"])]
        return casa
    except:
        return data

In [9]:
calle = "Calle la Maso 1 28034 Madrid"

In [10]:
geocode(calle)

[40.49658, -3.70491]

#### Primer ejemplo

In [35]:
calle = "Calle Marbella 83 28034, Madrid"

In [36]:
data = requests.get(f"https://geocode.xyz/{calle}?json=1").json()
data

{'standard': {'stnumber': '83',
  'addresst': 'Marbella',
  'postal': '28034',
  'region': 'Comunidad de Madrid',
  'prov': 'ES',
  'city': 'Madrid',
  'countryname': 'Spain',
  'confidence': '0.8'},
 'longt': '-3.71037',
 'alt': {},
 'elevation': {},
 'latt': '40.49690'}

In [71]:
home = geocode(calle)
home

[40.4969, -3.71037]

In [48]:
home.keys()

dict_keys(['type', 'coordinates'])

#### Segundo ejemplo

In [11]:
calle = "Calle Nuñez Morgado 9, Madrid"

In [12]:
geocode(calle)

[40.4687, -3.68553]

####  Tercer ejemplo

In [13]:
calle = "Calle de Villanueva, 11 29001 Madrid"

In [17]:
coordenadas = geocode(calle)
coordenadas

[40.42253, -3.68632]

## JUGANDO CON FUNCIONES

### Calculamos con la calle dada por el paciente el centro medico más cercano

- La calle dada es en froma de string. Este string recorreria nuestra funcion formada por la API de Geocode.
- Posteriormente las coordenadas sacadas se introcuden en la funcion de calculo con todos los centros medicos y te devuelve el centro más cercano a tu casa

In [18]:
centros_medicos_clean = pd.read_csv("../data/centros_medicos_clean.csv")
centros_medicos_clean.head()

,Unnamed: 0,NOMBRE,DIRECCION,DISTRITO,COD_POSTAL,LATITUD,LONGITUD,TIPO
0,0,Centro Concertado de Atención a las Adicciones...,CALLE SANTA HORTENSIA 1,CHAMARTIN,28002,40.446053,-3.670277,CentrosAsistenciaDrogodependientes
1,1,Centro Concertado de Atención a las Adicciones...,AVENIDA PORTUGAL 0,LATINA,28011,40.416331,-3.723991,CentrosAsistenciaDrogodependientes
2,2,Centro de Atención a las Adicciones (CAD) de A...,PASEO CHOPERA 12,ARGANZUELA,28045,40.393057,-3.698067,CentrosAsistenciaDrogodependientes
3,3,Centro de Atención a las Adicciones (CAD) de H...,CALLE MINAYA 7,HORTALEZA,28043,40.468289,-3.636150,CentrosAsistenciaDrogodependientes
4,4,Centro de Atención a las Adicciones (CAD) de L...,CALLE CAMARENA 10,LATINA,28047,40.395971,-3.756272,CentrosAsistenciaDrogodependientes


In [19]:
calle = "Calle Tembleque, 52 28024 Madrid"

In [20]:
def geocode(calle):
    """
    Saca las coordenadas de una dirección que le des.
    """
    data = requests.get(f"https://geocode.xyz/{calle}?json=1").json()
    try:
        casa = [float(data["latt"]),float(data["longt"])]
        return casa
    except:
        return data

In [21]:
casa = geocode(calle)
casa

[40.4046, -3.71294]

In [22]:
def haversine(coord1, coord2):
    
    '''
    Funcion en donde al meterle dos parametros de coordenadas te va a devolver la distancia que hay entre ambos puntos
    '''
    
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [23]:
def distancia_entre_hospital_casa(casa):

    '''
    Esta funcion lo que coge es la latitud y longitud de la ubicacion de tu casa y te evuelve el nombre del centro medico 
    mas cercano a tu casa
    '''
    
    for index, row in centros_medicos_clean.iterrows(): 
        row['LATITUD']
        row['LONGITUD']
    
    centros_medicos_clean['DISTANCIA'] = 0 
    
    centros_medicos_clean['DISTANCIA'] = centros_medicos_clean.apply(lambda row: haversine((casa), (row['LATITUD'], row['LONGITUD'])), axis=1)
    
    return (centros_medicos_clean.loc[centros_medicos_clean.DISTANCIA.argmin(), "NOMBRE"], math.trunc(centros_medicos_clean.loc[centros_medicos_clean.DISTANCIA.argmin(), "DISTANCIA"]))
    

In [24]:
distancia_entre_hospital_casa(casa)

('CMSc Centro Municipal de Salud Comunitaria Arganzuela', 216)

###  Cálculo de la farmacia más cercana a tu casa

- Introducioendo el strind de la dirreccion de tu casa en la funcion creada por la API geocode sacaremos las coordenadas de tu casa.
- Posterirmente estas coordenadas las meteremos en otra función calculando asi la distancia con todas las farmacias de Madrid y devolviendo la farmacia más cercana a tu casa

In [25]:
farmacias_clean = pd.read_csv("../data/farmacias_clean.csv")
farmacias_clean.head()

,Unnamed: 0,titular_nombre,direccion,municipio_nombre,latitud,longitud
0,0,Amaya Corral Palomino,PLAZA del Dos de Mayo 6,Madrid,40.427326,-3.704186
1,1,Montserrat Gimenez Mestre,CALLE de Ríos Rosas 50,Madrid,40.441590,-3.696234
2,2,Justo Aquilino Corral Aragon,CALLE de Argensola 12,Madrid,40.425284,-3.694781
3,3,Irene Arroita Del Campo,CALLE de Alcalá 98,Madrid,40.424996,-3.674844
4,4,Esther Sansi Ponsico,PLAZA de Salesas 7,Madrid,40.423736,-3.694447


In [26]:
casa = geocode(calle)
casa

[40.4046, -3.71294]

In [27]:
def distancia_entre_farmacia_casa(casa):
    for index, row in farmacias_clean.iterrows(): 
        row['latitud']
        row['longitud']
    
    farmacias_clean['DISTANCIA'] = 0 
    
    farmacias_clean['DISTANCIA'] = farmacias_clean.apply(lambda row: haversine(casa, (row['latitud'], row['longitud'])), axis=1)
    
    return (farmacias_clean.loc[farmacias_clean.DISTANCIA.argmin(), "titular_nombre"], farmacias_clean.loc[farmacias_clean.DISTANCIA.argmin(), "DISTANCIA"])

In [28]:
distancia_entre_farmacia_casa(casa)

('Maria Jose Muñoz Moya', 40.45691826489327)

### Mapa para visualizar las 3 localizaciones recibidas:
    - Direccioón de tu casa
    - Dirección de Hospital más cercano
    - Dirección de la farmacia más cercana

#### Función que saca las coordenadas de la farmacia más cercana a la dirreción que se haya introducido

In [64]:
def coordenadas_farmacia_mas_cercana(casa):
    for index, row in farmacias_clean.iterrows(): 
        row['latitud']
        row['longitud']
     
    farmacias_clean['DISTANCIA'] = 0
    farmacias_clean['DISTANCIA'] = farmacias_clean.apply(lambda row: haversine(casa, (row['latitud'], row['longitud'])), axis=1)
    
    return (farmacias_clean.loc[farmacias_clean.DISTANCIA.argmin(), "latitud"], farmacias_clean.loc[farmacias_clean.DISTANCIA.argmin(), "longitud"])

In [65]:
coord_farm_cercana = coordenadas_farmacia_mas_cercana(casa)
coord_farm_cercana

(40.40494855569637, -3.7128034492139332)

#### Función que saca las coordenadas del hospital más cercano a la dirección que se haya introducido

In [66]:
def coordenadas_hospital_mas_cercano(casa):

    '''
    Esta funcion lo que coge es la latitud y longitud de la ubicacion de tu casa y te evuelve el nombre del centro medico 
    mas cercano a tu casa
    '''
    
    for index, row in centros_medicos_clean.iterrows(): 
        row['LATITUD']
        row['LONGITUD']
    
    centros_medicos_clean['DISTANCIA'] = 0 
    
    centros_medicos_clean['DISTANCIA'] = centros_medicos_clean.apply(lambda row: haversine((casa), (row['LATITUD'], row['LONGITUD'])), axis=1)
    
    return (centros_medicos_clean.loc[centros_medicos_clean.DISTANCIA.argmin(), "LATITUD"], (centros_medicos_clean.loc[centros_medicos_clean.DISTANCIA.argmin(), "LONGITUD"]))
    

In [67]:
coord_hospital_cercano = coordenadas_hospital_mas_cercano(casa)
coord_hospital_cercano

(40.4056915626929, -3.71081961282155)

#### Mapa con las tres variables de coordenadas sacadas

In [68]:
# VARIABLES DE LOS ICONOS (en visual no tienes que ponerlos porque ya estan definidos)
path = "../icons/{}".format

icon_hosp = path("Hospital-clinica-sanatorio-icon.png")
icon_farm = path("cruz-icon.png")

In [69]:
# Las printeo solamente para ver que no hay ningun error
print(casa)
print(coord_farm_cercana)
print(coord_hospital_cercano)

[40.4046, -3.71294]
(40.40494855569637, -3.7128034492139332)
(40.4056915626929, -3.71081961282155)


In [70]:
def distancia_visual(casa):
    coord_farm_cercana = coordenadas_farmacia_mas_cercana(casa)
    coord_hospital_cercano = coordenadas_hospital_mas_cercano(casa)
    map_final = Map(location = casa, zoom_start=15, control_scale = True)
    #HOSPiTAL
    icon = CustomIcon( 
            icon_hosp, 
            icon_size=(40, 50))
    
    loc = coord_hospital_cercano
    hospital_cercano = Marker(location = loc, icon = icon, tooltip = "El hospital mas cercano")
    hospital_cercano.add_to(map_final)


    #FARMACIA
    icon = CustomIcon(
            icon_farm,
            icon_size=(40, 50))

    loc = coord_farm_cercana
    farmacia_cercana = Marker(location = loc,icon = icon, tooltip = "La farmacia mas cercana")
    farmacia_cercana.add_to(map_final)

    #HOME
    icon = Icon(color = "red",
                prefix = "fa",
                icon = "home",
                icon_color = "white",
                icon_size = (40, 50))
    loc = casa
    mi_casa = Marker(location = loc, icon = icon, tooltip = "Tu casa")
    mi_casa.add_to(map_final)


    return map_final

In [71]:
distancia_visual(map_final)

TypeError: cannot unpack non-iterable Map object

In [72]:
map_final = Map(location = casa, zoom_start=15, control_scale = True)

#HOSPiTAL
icon = CustomIcon( 
            icon_hosp, 
            icon_size=(40, 50))
loc = coord_hospital_cercano

hospital_cercano = Marker(location = loc, icon = icon, tooltip = "El hospital mas cercano")
hospital_cercano.add_to(map_final)


#FARMACIA
icon = CustomIcon(
            icon_farm,
            icon_size=(40, 50))

loc = coord_farm_cercana
farmacia_cercana = Marker(location = loc,icon = icon, tooltip = "La farmacia mas cercana")
farmacia_cercana.add_to(map_final)


#HOME
icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "home",
                    icon_color = "white",
                    icon_size = (40, 50))
loc = casa
mi_casa = Marker(location = loc, icon = icon, tooltip = "Tu casa")
mi_casa.add_to(map_final)


map_final

In [75]:
map_final = Map(location = casa, zoom_start=15, control_scale = True)

def distancia_visual(map_final):
    map_final = Map(location = casa, zoom_start=15, control_scale = True)
    #HOSPiTAL
    icon = CustomIcon( 
                icon_hosp, 
                icon_size=(40, 50))
    loc = coord_hospital_cercano

    hospital_cercano = Marker(location = loc, icon = icon, tooltip = "El hospital mas cercano")
    hospital_cercano.add_to(map_final)


    #FARMACIA
    icon = CustomIcon(
                icon_farm,
                icon_size=(40, 50))

    loc = coord_farm_cercana
    farmacia_cercana = Marker(location = loc,icon = icon, tooltip = "La farmacia mas cercana")
    farmacia_cercana.add_to(map_final)


    #HOME
    icon = Icon(color = "red",
                        prefix = "fa",
                        icon = "home",
                        icon_color = "white",
                        icon_size = (40, 50))
    loc = casa
    mi_casa = Marker(location = loc, icon = icon, tooltip = "Tu casa")
    mi_casa.add_to(map_final)


    return map_final

In [76]:
distancia_visual(map_final)

In [52]:
map_final_2 = Map(location = casa, zoom_start=15, control_scale = True)

def distancia_visual(map_final_2):
    
    #FARMACIA
    icon = CustomIcon(
                icon_farm,
                icon_size=(40, 50))

    loc = coord_farm_cercana
    farmacia_cercana = Marker(location = loc,icon = icon, tooltip = "La farmacia mas cercana")
    farmacia_cercana.add_to(map_final_2)


    #HOME
    icon = Icon(color = "red",
                        prefix = "fa",
                        icon = "home",
                        icon_color = "white",
                        icon_size = (40, 50))
    loc = casa
    mi_casa = Marker(location = loc, icon = icon, tooltip = "Tu casa")
    mi_casa.add_to(map_final_2)


    return map_final_2

In [53]:
distancia_visual(map_final_2)